# Redis

# Race condition?

* A race condition is a situation in which two or more parts of a software are reading or writing shared data, and the final result.
* depends on which process finishes or starts first. Race conditions happen when the order of execution is not guaranteed; that's why it's called a "race".

Here are **clear, interview-ready explanations** of **Rate Limiting** and **TTL**, with Redis + Java context.

---

## 1️⃣ What is Rate Limiting?

**Rate limiting** is a technique to **control how many requests a user, IP, or client can make within a fixed time window**.

### Simple definition

> Rate limiting restricts the number of actions allowed per user in a given time to prevent abuse, overload, or unfair usage.

---

### Why rate limiting is needed

* Prevent **API abuse / DDoS**
* Protect **backend & database**
* Ensure **fair usage**
* Control **cost** (external APIs, cloud resources)

---

### Real-world examples

| Scenario       | Limit                   |
| -------------- | ----------------------- |
| Login attempts | 5 per minute            |
| OTP requests   | 3 per 10 minutes        |
| Public API     | 100 requests per minute |
| Payment API    | 1 request per second    |

---

### Rate limiting using Redis (most common)

Redis is ideal because:

* Extremely fast
* Atomic operations (`INCR`)
* Supports TTL automatically

---

### Basic Redis rate-limiting logic

1. User makes request
2. Create a Redis key:

   ```
   rate:user123:minute
   ```
3. Increment counter:

   ```
   INCR rate:user123:minute
   ```
4. If first request → set TTL = 60 seconds
5. If count > limit → reject request

---

### Why Redis works well

* `INCR` is **atomic** (thread-safe)
* TTL auto-resets counter
* Shared across all app instances

---

### Rate limiting types (interview)

| Type           | Description                |
| -------------- | -------------------------- |
| Fixed window   | X requests per time window |
| Sliding window | More accurate, smoother    |
| Token bucket   | Allows bursts              |
| Leaky bucket   | Smooth constant rate       |

👉 **Redis usually uses Fixed or Sliding window**

---

### Interview answer (short)

> Rate limiting is a mechanism to restrict how many requests a client can make in a given time window, commonly implemented using Redis counters with TTL.

---

## 2️⃣ What is TTL (Time To Live)?

**TTL** defines **how long a key exists in Redis before it expires automatically**.

### Simple definition

> TTL is the lifespan of a key after which Redis automatically deletes it.

---

### Why TTL is important

* Prevent stale data
* Free memory automatically
* Reset counters (rate limiting)
* Avoid manual cleanup

---

### TTL examples

| Use case           | TTL          |
| ------------------ | ------------ |
| Cache data         | 5–30 minutes |
| OTP                | 2–5 minutes  |
| Session            | 30 minutes   |
| Rate limit counter | 1 minute     |

---

### TTL in Redis

* Set during write:

  ```
  SET key value EX 60
  ```
* Or later:

  ```
  EXPIRE key 60
  ```

Redis deletes the key **automatically after TTL expires**.

---

### TTL behavior

* TTL countdown starts immediately
* When TTL reaches zero → key removed
* Accessing key does NOT reset TTL (unless explicitly updated)

---

### TTL with Rate Limiting (key concept)

TTL defines the **rate-limit window**.

Example:

```
rate:user123 → value=5, TTL=60s
```

After 60 seconds:

* key disappears
* counter resets
* user can make requests again

---

## 3️⃣ Rate Limiting + TTL together (very important)

### Example: 5 requests per minute

| Time | Redis state                 |
| ---- | --------------------------- |
| 00s  | rate:user1 = 1 (TTL 60s)    |
| 10s  | rate:user1 = 2              |
| 30s  | rate:user1 = 5              |
| 40s  | ❌ request rejected          |
| 60s  | key expires                 |
| 61s  | rate:user1 = 1 (new window) |

👉 TTL automatically resets the window.

---

## 4️⃣ Interview-ready comparison

### Rate Limiting vs Throttling

| Rate Limiting    | Throttling            |
| ---------------- | --------------------- |
| Hard stop        | Slows down requests   |
| Rejects requests | Delays requests       |
| Security focused | Performance smoothing |

---

## 5️⃣ Common interview traps (watch out)

❌ Forgetting TTL → counter never resets
❌ Using in-memory counter → breaks in multi-instance apps
❌ Resetting TTL on every request (wrong for fixed window)

---

## 6️⃣ One-line interview answers

**Rate Limiting**

> A technique to restrict the number of requests per user or client within a specific time window.

**TTL**

> The duration after which a key automatically expires and is removed from Redis.

---

## 7️⃣ Where this is used in Java apps

* API gateways
* Login & OTP protection
* Payment APIs
* Public REST endpoints
* Microservices

---

If you want next:

* 🔹 Redis rate limiter **Java code**
* 🔹 Sliding window vs fixed window
* 🔹 Spring Boot filter/interceptor example
* 🔹 Interview coding question with solution

Just tell me 👍
